In [1]:
import numpy as np

from collections import Counter, defaultdict

import json
from xgboost import XGBClassifier
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

from feature_extraction import *
from classify import *
from book import *

In [2]:
from evaluation import *

ann_comb, ann_SOC, ann_ASOIAF, ann_SA
lengths:  [622, 91, 256, 275]
POVs:  30 9 15 6


In [3]:
from feature_extraction_spacy import *

# Sanity Check

In [48]:
mdl = CL_mdl()
mdl.train(*extract_texts_and_characters(ann_ASOIAF))

In [49]:
texts, reference_chars = extract_texts_and_characters(ann_ASOIAF)
output_chars = list(mdl.choose_characters(texts))


In [62]:
err_inds = output_chars!=reference_chars
np.mean(err_inds)

0.0703125

In [70]:
len())

18

In [76]:
list(map(lambda x:list(zip(*x)), map(mdl.character_scores, texts[err_inds])))

[[(0.12312941, 'Hodor'),
  (0.009316656, 'Ser Rodrik'),
  (0.0009677459, 'Winterfell'),
  (0.0013692211, 'Robb'),
  (0.0048291446, 'Rickon'),
  (0.0057737306, 'Wyman'),
  (0.002581664, 'Maester Luwin'),
  (0.0028235833, 'Howland Reed'),
  (0.0014790919, 'Stark'),
  (0.0035893538, 'Joseth'),
  (0.0070313644, 'Jojen'),
  (0.000925843, 'Meera'),
  (0.0057115885, 'Robert'),
  (0.027417276, 'Bran'),
  (0.005922722, 'Arya'),
  (0.008172362, 'Jon'),
  (0.0014429394, 'House'),
  (0.002204129, 'Greywater'),
  (0.00082771113, 'Osha'),
  (0.0010652458, 'Cley Cerwyn'),
  (0.0012111345, 'Beth'),
  (0.005536692, 'Mikken'),
  (0.0013958206, 'Lady Hornwood'),
  (0.004205637, 'Hayhead'),
  (0.0034541404, 'Neck'),
  (0.0039961594, 'Walder'),
  (0.0027344157, 'Uncle Benjen'),
  (0.0025302893, 'Wheels'),
  (0.0010690439, 'Princess Myrcella'),
  (0.0040371995, 'Eddard'),
  (0.0017861172, 'Harwin'),
  (0.0028232543, 'Reeds'),
  (0.0006842509, 'Reed'),
  (0.0014611245, 'Old Nan'),
  (0.00060374476, 'Umbers')

In [64]:
np.vstack([output_chars, reference_chars])[:,err_inds].T


array([['Hodor', 'Bran'],
       ['Cersei', 'Tyrion'],
       ['Stannis', 'Tyrion'],
       ['Littlefinger', 'Eddard'],
       ['Cersei', 'Tyrion'],
       ['Maester Luwin', 'Bran'],
       ['Alebelly', 'Bran'],
       ['Hodor', 'Bran'],
       ['Hodor', 'Bran'],
       ['Gilly', 'Samwell'],
       ['Meera', 'Bran'],
       ['Walder', 'Bran'],
       ['Hodor', 'Bran'],
       ['Robb', 'Bran'],
       ['Theon', 'Bran'],
       ['Robb', 'Bran'],
       ['Hodor', 'Bran'],
       ['Jojen', 'Bran']], dtype='<U13')

# Evaluation Program

In [31]:
import pandas as pd

def all_metrics(tt,pp):
    #prf = precision_recall_fscore_support(tt,pp, average='micro', labels=np.unique(tt))[0:3]
    acc = accuracy_score(tt,pp)
    return acc #np.hstack([prf, acc])

all_metrics_names = ["Acc"]

# Eval

In [32]:
WE_mdl = lambda: MLCharacterSolver(XGBClassifier(), nicknames2name_comb, get_spacy_embedding_features)
CL_mdl = lambda: MLCharacterSolver(XGBClassifier(), nicknames2name_comb, get_spacy_feature_vectors)
HY_mdl = lambda: MLCharacterSolver(XGBClassifier(), nicknames2name_comb, hybrid_features)


FM_mdl = lambda: FirstMentionedSolver(nicknames2name_comb)
MC_mdl = lambda: MostMentionedSolver(nicknames2name_comb)

datasets = [("ASIAF", ann_ASOIAF), ("SOC", ann_SOC)]
supdatasets = [("SA", ann_SA)]
base_mdls = [#("ML Classical Features", CL_mdl)
            ("ML Word Emb. Features", WE_mdl)
]

## main eval

In [33]:
def make_program(datasets, mdls):
    program = dict()    
    for (test_data_name, test_data),(mdl_name,mdl) in it.product(datasets, mdls):
        if mdl_name[0:2]=="ML":
            for (train_data_name, train_data) in datasets:
                if train_data_name==test_data_name:
                    continue
                program[(test_data_name,mdl_name, train_data_name)] = (
                    train_data,
                    test_data,
                    mdl()
                )
        else:
            program[(test_data_name, mdl_name, "---")] = ([], test_data, mdl())
    return program

program = make_program(datasets, base_mdls)


res = pd.DataFrame(index=pd.MultiIndex.from_tuples(program.keys()),
                   columns = all_metrics_names)
res.sort_index(inplace=True)

for ind in res.index:
    print(ind, end="")
    score = evaluate(*program[ind], metric=all_metrics)
    res.loc[ind,:] = score
    print(" ", score)
    res.to_csv("../results/maineval_spacy.csv", index_label=["Test Set", "Method", "Train Set"])
    
res

('ASIAF', 'ML Word Emb. Features', 'SOC')  0.8359375
('SOC', 'ML Word Emb. Features', 'ASIAF')  0.9010989010989011


,,,Acc
ASIAF,ML Word Emb. Features,SOC,0.835938
SOC,ML Word Emb. Features,ASIAF,0.901099


In [7]:
# spacy.load('en_core_web_lg', disable=['parser'])
('ASIAF', 'ML Classical Features', 'SOC')  0.8984375
('SOC', 'ML Classical Features', 'ASIAF')  0.8241758241758241
('ASIAF', 'ML Word Emb. Features', 'SOC')  0.8359375
('SOC', 'ML Word Emb. Features', 'ASIAF')  0.9010989010989011

# spacy.load('en_core_web_sm', disable=['parser'])
('ASIAF', 'ML Classical Features', 'SOC')  0.8515625
('SOC', 'ML Classical Features', 'ASIAF')  0.8351648351648352
